<a href="https://colab.research.google.com/github/naterattner/data71200/blob/master/project_3/data71200_summer24_project3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Python notebook for project 3: https://bbhosted.cuny.edu/ultra/courses/_2383576_1/cl/outline